In [1]:
import tensorflow as tf
import numpy as np

In [2]:
with np.load("notMNIST.npz") as data:
    Data, Target = data ["images"], data["labels"]
    Data = np.reshape(Data, [-1, 28*28])
    np.random.seed(521)
    randIndx = np.arange(len(Data))
    np.random.shuffle(randIndx)
    Data = Data[randIndx]/255.
    Target = Target[randIndx]
    trainData, trainTarget = Data[:15000], Target[:15000]
    validData, validTarget = Data[15000:16000], Target[15000:16000]
    testData, testTarget = Data[16000:], Target[16000:]

In [3]:
num_label = 10
num_feature = 28*28
train_size = len(trainData)

In [4]:
with tf.Session() as s:
    testTarget_oh = s.run(tf.one_hot(testTarget, num_label))
    trainTarget_oh = s.run(tf.one_hot(trainTarget, num_label))
    validTarget_oh = s.run(tf.one_hot(validTarget, num_label))

In [5]:
#TODO: add bias
#TODO: weight decay
def build_layer(prev_layer, num_node):
    prev_num_node = prev_layer.shape[1].value
    xavier_stddev = tf.sqrt(3.0/(prev_num_node + num_node))
    W = tf.Variable(tf.random_normal(shape=[prev_num_node, num_node], stddev=xavier_stddev))
    b = tf.Variable(tf.zeros([num_node]))
    
    return tf.add(tf.matmul(prev_layer, W), b)

In [6]:
x = tf.placeholder("float", shape=[None, num_feature])
y_true = tf.placeholder("float", shape=[None, num_label])

In [7]:
layer1 = build_layer(x, 1000)
layer1 = tf.nn.relu(layer1)

layer2 = build_layer(layer1, 1000)
layer2 = tf.nn.relu(layer2)

layer3 = build_layer(layer2, 1000)
layer3 = tf.nn.relu(layer3)

layer4 = build_layer(layer3, 1000)
layer4 = tf.nn.relu(layer4)

layer5 = build_layer(layer4, 1000)
layer5 = tf.nn.relu(layer5)

In [8]:
y = build_layer(layer3, num_label) #output layer
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_true, logits=y))

In [9]:
learning_rate = 0.005
train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [10]:
batch_size = 500
epoch_ratio = int(train_size/batch_size)
iteration = 20000

In [11]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_true, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [13]:
with tf.Session() as s:
    tf.global_variables_initializer().run()
    
    for step in xrange(iteration):
        offset = (step*batch_size) % train_size
        
        batch_data = trainData[offset:(offset + batch_size)]
        batch_target = trainTarget_oh[offset:(offset + batch_size)]
    
        train_step.run(feed_dict={x: batch_data, y_true: batch_target})
    
        if step%(epoch_ratio)==0:
            randIndx = np.arange(len(trainData))
            np.random.shuffle(randIndx)
            trainData, trainTarget = trainData[randIndx], trainTarget[randIndx]
            trainTarget_oh = s.run(tf.one_hot(trainTarget, num_label))
        
        if step%(epoch_ratio * 100) == 0:
            print s.run(loss, feed_dict={x: trainData, y_true: trainTarget_oh})
            print accuracy.eval({x: testData, y_true: testTarget_oh})
            print ''

43.147808
0.10462555

0.013762262
0.91997063

0.0025197566
0.9229075

0.21031204
0.8986784

0.020712024
0.9232746

0.02065319
0.9229075

0.014427692
0.9192364

